In [63]:
import numpy as np
from typing import Callable, Union, Tuple, List, Any

# Initial-Value Problems for Ordinary Differential Equations

## Chapter 5.2 Euler's Method
Given 
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
and consider
$$ t_i = a+ih,\,\,\,i=0,1,2\cdots,N$$
Suppose $y(t)$ has two continuous derivatives on $[a, b]$, then by Taylor's Theorem
$$ y(t_{i+1})=y(t_i)+(t_{i+1}-t_i)y'(t_i)+\frac{(t_{i+1}-t_i)^2}{2}y''(\xi_i) $$
because $h=t_{i+1}-t_i$, then
$$ y(t_{i+1})=y(t_i)+hf(t_i,y(t_i))+\frac{h^2}{2}y''(\xi_i)$$

The definition of $t_i$ and $h$ are the same throughout the context.

Euler's method constructs $w_i\approx y(t_i)$, by deleting the remainder term. In short,
$$w_0=\alpha$$
$$w_{i+1}=w_i+hf(t_i, w_i),\,\,\,i=0, 1,\cdots,N-1$$

In [64]:
def Euler(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          alpha:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * f(t_[idx - 1], w_[idx - 1])
    return (t_, w_)

In [65]:
f = lambda t, y: y - t ** 2 + 1
Euler(f, 0, 2, 0.5, h=0.5)

(array([0. , 0.5, 1. , 1.5, 2. ]),
 array([0.5   , 1.25  , 2.25  , 3.375 , 4.4375]))

In [66]:
f = lambda t, y: np.cos(2 * t) + np.sin(3 * t)
Euler(f, 0, 1, 1, N=4)

(array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
 array([1.        , 1.25      , 1.63980533, 2.02425465, 2.23645725]))

## Chapter 5.3 Higher-Order Taylor Methods
Suppose the solution $y(t)$ to the initial-value problem
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
has $(n+1)$ continuous derivatives. By its $n$th Taylor polynomial about $t_i$ and evaluate at $t_{i+1}$,
$$y(t_{{i+1}})=y(t_i)+hy'(t_i)+\frac{h^2}{2}y''(t_i)+\cdots+\frac{h^n}{n!}y^{(n)}(t_i)+\frac{h^{n+1}}{(n+1)!}y^{(n+1)}(\xi_i)$$
for some $\xi_i\in(t_i, t_{i+1})$
Successive differentiation of the solution, $y(t)$ gives
$$y'(t)=f(t, y(t)),\,\,\,y''(t)=f'(t, y(t)),\,\,\,\text{ and generally, }\,\,\, y^{(k)}(t)=f^{(k-1)}(t, y(t))$$
Substituting these findings into previous equations gives
$$y(t_{{i+1}})=y(t_i)+hf(t_i)+\frac{h^2}{2}f'(t_i)+\cdots+\frac{h^n}{n!}f^{(n - 1)}(t_i)+\frac{h^{n+1}}{(n+1)!}f^{(n)}(\xi_i)$$
Similar to Euler's method, in short
$$w_0=\alpha$$
$$w_{i+1}=w_i+hT^{(n)}(t_i, w_i),\,\,\,i=0,1,\cdots,N-1$$
where
$$T^{(n)}(t_i, w_i)=\sum^{n-1}_{i=0}\frac{h^{i}}{(i+1)!}f^{(i)}(t_i, w_i)$$
Euler's method is Taylor's method of order one.

In [67]:
def Taylor(func:List[Callable[[Union[float, int], Union[float, int]], Union[float, int]]],
           a:Union[float, int],
           b:Union[float, int],
           alpha:Union[float, int],
           h:Union[float, int]=None,
           N:int=None
          ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha

    factorial = [1] * (len(func) + 1)
    for idx in range(2, len(func) + 1):
        factorial[idx] = factorial[idx - 1] * (idx)
    for idx in range(1, N + 1):
        T = sum([(h ** jdx) * f(t_[idx - 1], w_[idx - 1]) / factorial[jdx + 1] for jdx, f in enumerate(func)])
        w_[idx] = w_[idx - 1] + h * T
    return (t_, w_)

In [68]:
f = [
    lambda t, y: y - t ** 2 + 1,
    lambda t, y: y - t ** 2 + 1 - 2 * t
]
Taylor(f, 0, 2, 0.5, 0.2)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.83      , 1.2158    , 1.652076  , 2.13233272,
        2.64864592, 3.19134802, 3.74864458, 4.30614639, 4.8462986 ,
        5.34768429]))

## Chapter 5.4 Runge-Kutta Methods
__Runge-Kutta methods__ have the high-order local truncation error of the Taylor methods
but eliminate the need to compute and evaluate the derivatives of $f (t, y)$

In general, __Runge-Kutta Methods__ have the following form:
$$
y_{i+1} = y_i+h\sum_{i=1}^{s}b_ik_i
$$
where
$y_i=y(t_i)$

$k_1=f(t_i,y_i)$

$k_2=f(t_i+c_2h,y_i+(a_{21}k_1)h)$

$k_3=f(t_i+c_3h,y_i+(a_{31}k_1+a_{32}k_2)h)$

$\vdots$

$k_s=f(t_i+c_sh,y_i+(a_{s1}k_1+a_{s2}k_2+\cdots+a_{s, s-1})h)$

The parameters above can be store as an matrix where the illustrated as below:
\begin{bmatrix}
0 & 0 & 0 & \cdots & 0\\
c_2 & a_{21} & a_{22} & \cdots & a_{s2}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
c_s & a_{s1} & a_{s2} & \cdots & a_{ss}\\
0 & b_1 & b_2 & \cdots & b_s
\end{bmatrix}	

Refer to [Wikipedia](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for more information

### Runge-Kutta Methods of Order Two
There is a family of such methods, parameterized by α and given by the formula
$$
y(t_{i+1})=y(t_i)+h\biggl(\biggl(1-\frac{1}{2\alpha}f(t_i, y(t_i))\biggr)+\frac{1}{2\alpha}f(t_i+\alpha h, y(t_i)+\alpha f(t_i, y(t_i)))\biggr)
$$
where 
$\alpha=\frac{1}{2}$ gives the midpoint method, $\alpha=1$ is Modified Euler Method(Heun's method), and $\alpha=\frac{2}{3}$ is Ralston's method.

So by the matrix notation introduced above, we have
\begin{bmatrix}
0 & 0 & 0\\
\alpha & \alpha & 0\\
0 & 1 - \frac{1}{2\alpha} & \frac{1}{2\alpha}
\end{bmatrix}	


#### Midpoint Method
$w_0=\alpha$

$w_{i+1}=w_i+hf\biggl(t_i+\frac{h}{2}, w_i+\frac{h}{2}f(t_i, w_i)\biggr),\,\,\,i=0, 1,\cdots,N-1$

#### Modified Euler Method
$w_0=\alpha$

$w_{i+1}=w_i+\frac{h}{2}[f(t_i, w_i)+f(t_{i+1}, w_i+hf(t_i, w_i))],\,\,\,i=0, 1,\cdots,N-1$

#### Ralston's Method
$w_0=\alpha$

$w_{i+1}=w_i+h\biggl[\frac{1}{4}f(t_i, w_i)+\frac{3}{4}f(t_{i+1}+\frac{2}{3}h, w_i+\frac{2}{3}hf(t_i, w_i))\biggr],\,\,\,i=0, 1,\cdots,N-1$

In [69]:
def RK2(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          y_init:Union[float, int],
          h:Union[float, int]=None,
          alpha:float=0.5,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * ((1 - 1 / (2 * alpha)) * f(t_[idx - 1], w_[idx - 1]) + 
                                      (f(t_[idx - 1] + alpha * h, w_[idx - 1] + alpha * h * f(t_[idx - 1], w_[idx - 1]))) / (2 * alpha)
                                      )
    return (t_, w_)

In [70]:
import pandas as pd
f = lambda t, y: y - t ** 2 + 1

t_1, w_1 = RK2(f, 0, 2, 0.5, 0.2, alpha=0.5)
t_2, w_2 = RK2(f, 0, 2, 0.5, 0.2, alpha=1)
t_3, w_3 = RK2(f, 0, 2, 0.5, 0.2, alpha=2 / 3)

pd.DataFrame(
    {
        "t":t_1,
        "Midpoint":w_1,
        "Modified Euler":w_2,
        "Ralston":w_3
    }
)


,t,Midpoint,Modified Euler,Ralston
0,0.0,0.500000,0.500000,0.500000
1,0.2,0.828000,0.826000,0.827333
2,0.4,1.211360,1.206920,1.209880
3,0.6,1.644659,1.637242,1.642187
4,0.8,2.121284,2.110236,2.117601
5,1.0,2.633167,2.617688,2.628007
6,1.2,3.170463,3.149579,3.163502
7,1.4,3.721165,3.693686,3.712006
8,1.6,4.270622,4.235097,4.258780
9,1.8,4.800959,4.755619,4.785845


### Runge-Kutta Methods of Order Three
#### Heun's Method

$w_0=\alpha$

$w_{i+1}=w_i+\frac{h}{4}\biggl[f(t_i, w_i)+3f(t_{i}+\frac{2}{3}h, w_i+\frac{2}{3}hf(t_i+\frac{1}{3}h, w_i+\frac{1}{3}f(t_i, w_i)))\biggr],\,\,\,i=0, 1,\cdots,N-1$

In [71]:
def Heun(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          y_init:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * y_init
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * (f(t_[idx - 1], w_[idx - 1]) + 3 * f(t_[idx - 1] + (2 / 3) * h, w_[idx - 1] + (2 / 3) * 
                                        h * f(t_[idx - 1] + (1 / 3) * h, w_[idx - 1] + (1 / 3) * h *  f(t_[idx - 1], w_[idx - 1])))) / 4
    return (t_, w_)

In [72]:
f = lambda t, y: y - t ** 2 + 1
t_, w_ = Heun(f, 0, 2, 0.5, 0.2)
t_, w_

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.82924444, 1.21397499, 1.6487659 , 2.12699053,
        2.64055555, 3.17957629, 3.73198028, 4.28302303, 4.81469657,
        5.30500719]))

# 